[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Wp-Zhang/HandyRec/blob/master/examples/YouTubeDNN.ipynb)

> This notebook runs YouTubeDNN including candidate generation and ranking on MovieLens1M dataset. 

> Only movies with ratings larger than 3 are treated as 'positive' samples for each user. Every last 10 'positive' movies of each user are held out for testing.

## Table of Contents:
* [Prepare data for matching](#section-0)
* [Train match model and export embeddings](#section-1)
* [Use Faiss to generate candidates](#section-2)
* [Train rank model and predict](#section-3)

**Download dataset and install packages**

In [1]:
! git clone https://github.com/Wp-Zhang/HandyRec.git
! pip install faiss-cpu

Cloning into 'HandyRec'...
remote: Enumerating objects: 994, done.
remote: Counting objects: 100% (994/994), done.
remote: Compressing objects: 100% (776/776), done.
remote: Total 994 (delta 336), reused 768 (delta 171), pack-reused 0
Receiving objects: 100% (994/994), 16.36 MiB | 8.93 MiB/s, done.
Resolving deltas: 100% (336/336), done.
     |████████████████████████████████| 8.6 MB 3.8 MB/s 


In [2]:
! wget https://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! unzip -o ml-1m.zip

--2022-03-26 01:38:26--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  4.16MB/s    in 1.4s    

2022-03-26 01:38:28 (4.16 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [3]:
!pip install -U tensorboard_plugin_profile

     |████████████████████████████████| 1.1 MB 4.3 MB/s 


**Import relative packages**

In [4]:
import sys
sys.path.append('./HandyRec/')

In [5]:
from handyrec.dataset.movielens import MovieMatchDataHelper, MovieRankDataHelper
from handyrec.layers.utils import sampledsoftmaxloss
from handyrec.models.match import YouTubeMatchDNN
from handyrec.models.rank import YouTubeRankDNN
from handyrec.features import DenseFeature, SparseFeature, SparseSeqFeature
from handyrec.dataset.metrics import map_at_k, recall_at_k
from handyrec.models.utils import search_embedding

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.losses import binary_crossentropy
import numpy as np
import pandas as pd
import gc

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
MATCH_EMBEDDING_DIM = 128
RANK_EMBEDDING_DIM = 128
SEQ_LEN = 40
BATCH_SIZE = 2**12
NEPOCH = 50

NEG_NUM = 10
CANDIDATE_NUM = 100

# 0. Prepare data for matching<a name="section-0"></a>

In [8]:
match_dh = MovieMatchDataHelper('./ml-1m/')
match_user_features = ['user_id','gender','age','occupation','zip']
match_movie_features = ['movie_id']

data = match_dh.get_clean_data(sparse_features=['gender','age','occupation','zip','year'])
match_dh.gen_dataset(match_user_features+match_movie_features, data, seq_max_len=SEQ_LEN, negnum=0)

100%|██████████| 4/4 [00:01<00:00,  3.88it/s]
0it [00:00, ?it/s]


In [9]:
match_train, match_train_label, match_test, match_test_label = match_dh.load_dataset(match_user_features, match_movie_features)

Load movie Features: 100%|██████████| 1/1 [00:00<00:00, 144.41it/s]


In [10]:
match_feature_dim = match_dh.get_feature_dim(data, match_user_features, match_movie_features, [])

# 1. Train match model and export embeddings <a name="section-1"></a>

In [11]:
# * add example_age^2 as showed in the original paper
match_train['example_age_2'] = match_train['example_age']**2
match_test['example_age_2'] = match_test['example_age']**2

In [12]:
match_user_dense_feats = ['example_age','example_age_2']
match_user_sparse_feats = ['user_id','gender','age','occupation','zip']

In [13]:
match_user_features = [SparseFeature(x, match_feature_dim[x], MATCH_EMBEDDING_DIM) for x in match_user_sparse_feats] +\
                [DenseFeature(x) for x in match_user_dense_feats] +\
                [SparseSeqFeature(SparseFeature('movie_id', match_feature_dim['movie_id'], MATCH_EMBEDDING_DIM), 'hist_movie_id',SEQ_LEN)]
match_item_id = SparseFeature('movie_id', match_feature_dim['movie_id'], MATCH_EMBEDDING_DIM)

In [14]:
match_model = YouTubeMatchDNN(
    match_user_features, match_item_id,
    dnn_hidden_units=(512,256,128,MATCH_EMBEDDING_DIM), 
    dnn_dropout=0.1,
    dnn_bn=True,
    num_sampled=100
)

In [15]:
match_model.compile(optimizer=tf.keras.optimizers.Adam(lr=5e-4), loss=sampledsoftmaxloss)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='./match_checkpoint/',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
history = match_model.fit(match_train, match_train_label,
                            batch_size=BATCH_SIZE, 
                            epochs=100,
                            verbose=1,
                            validation_split=0.1,
                            callbacks=[early_stop,checkpoint])

Epoch 1/100
207/207 [==============================] - 13s 34ms/step - loss: 3.7485 - val_loss: 3.5575
Epoch 2/100
207/207 [==============================] - 6s 30ms/step - loss: 3.0823 - val_loss: 3.1552
Epoch 3/100
207/207 [==============================] - 6s 31ms/step - loss: 2.7542 - val_loss: 2.7260
Epoch 4/100
207/207 [==============================] - 6s 31ms/step - loss: 2.5727 - val_loss: 2.4100
Epoch 5/100
207/207 [==============================] - 7s 32ms/step - loss: 2.4222 - val_loss: 2.3418
Epoch 6/100
207/207 [==============================] - 6s 31ms/step - loss: 2.2899 - val_loss: 2.2422
Epoch 7/100
207/207 [==============================] - 7s 31ms/step - loss: 2.1851 - val_loss: 2.2535
Epoch 8/100
207/207 [==============================] - 6s 30ms/step - loss: 2.1055 - val_loss: 2.1043
Epoch 9/100
207/207 [==============================] - 6s 30ms/step - loss: 2.0309 - val_loss: 2.1203
Epoch 10/100
207/207 [==============================] - 6s 31ms/step - loss: 1.96

In [16]:
match_model.load_weights('./match_checkpoint/')

In [17]:
all_item_model_input = {"movie_id": data['item']['movie_id'].values}

user_embedding_model = Model(inputs=match_model.user_input, outputs=match_model.user_embedding)
item_embedding_model = Model(inputs=match_model.item_input, outputs=match_model.item_embedding)

user_embs = user_embedding_model.predict(match_test, batch_size=2 ** 15)
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 15)

print(user_embs.shape)
print(item_embs.shape)

(6040, 128)
(3883, 128)


# 2. Use Faiss to generate candidates <a name="section-2"></a>

## Test match model

In [18]:
candidates = search_embedding(
    MATCH_EMBEDDING_DIM, 
    item_embs, 
    user_embs,
    data['item']['movie_id'].values,
    CANDIDATE_NUM)

In [19]:
map_at_k(match_test_label, candidates, k=10)

0.03653265925575528

In [20]:
recall_at_k(match_test_label, candidates, k=10)

0.08915562913907285

In [21]:
recall_at_k(match_test_label, candidates, k=100)

0.47059602649006627

## Prepare data for ranking

In [22]:
test_user_embs = user_embedding_model.predict(match_test, batch_size=2 ** 15)
test_candidates = search_embedding(
    MATCH_EMBEDDING_DIM, 
    item_embs, 
    test_user_embs,
    data['item']['movie_id'].values,
    CANDIDATE_NUM)

test_candidates = {
    match_test['user_id'][i] : test_candidates[i]
    for i in range(test_candidates.shape[0])
}

In [23]:
del user_embs, item_embs, match_train, match_train_label, test_user_embs
gc.collect()

50

In [24]:
rank_dh = MovieRankDataHelper('./ml-1m/')
rank_user_features = ['user_id','gender','age','occupation','zip']
rank_movie_features = [f for f in data['item'].columns if f != 'title']

rank_dh.gen_dataset(rank_user_features+rank_movie_features, data, test_candidates, seq_max_len=SEQ_LEN, negnum=NEG_NUM)

100%|██████████| 2/2 [00:13<00:00,  6.99s/it]


In [25]:
rank_train, rank_train_label, rank_test = rank_dh.load_dataset(rank_user_features, rank_movie_features)

Load movie Features: 100%|██████████| 3/3 [00:02<00:00,  1.10it/s]


In [26]:
rank_train['example_age_2'] = rank_train['example_age']**2
rank_test['example_age_2'] = rank_test['example_age']**2
rank_train['time_gap_2'] = rank_train['time_gap']**2
rank_train['time_gap_square'] = np.sqrt(rank_train['time_gap'])
rank_test['time_gap_2'] = rank_test['time_gap']**2
rank_test['time_gap_square'] = np.sqrt(rank_test['time_gap'])

In [27]:
rank_feature_dim = rank_dh.get_feature_dim(data, rank_user_features, rank_movie_features, [])

# 3. Train rank model and predict <a name="section-3"></a>

In [28]:
rank_user_dense_feats = ['time_gap','time_gap_2','time_gap_square','example_age','example_age_2']
rank_user_sparse_feats = ['user_id','gender','age','occupation', 'zip']
rank_item_dense_feats = []
rank_item_sparse_feats = [f for f in rank_movie_features if f!='genres']

In [29]:
rank_user_features = [SparseFeature(x, rank_feature_dim[x], RANK_EMBEDDING_DIM) for x in rank_user_sparse_feats] +\
                [DenseFeature(x) for x in rank_user_dense_feats] +\
                [SparseSeqFeature(SparseFeature('movie_id', rank_feature_dim['movie_id'], RANK_EMBEDDING_DIM), 'hist_movie_id',SEQ_LEN)]
rank_item_feats = [SparseFeature(x, rank_feature_dim[x], RANK_EMBEDDING_DIM) for x in rank_item_sparse_feats] +\
                [DenseFeature(x) for x in rank_item_dense_feats] +\
                [SparseSeqFeature(SparseFeature('genre_id', 19, MATCH_EMBEDDING_DIM), 'genres',6)]

In [30]:
rank_model = YouTubeRankDNN(
    rank_user_features, rank_item_feats, 
    dnn_hidden_units=(512,RANK_EMBEDDING_DIM), dnn_dropout=0.2
)

In [31]:
rank_model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4), loss=binary_crossentropy)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='./rank_checkpoint/',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
history = rank_model.fit(rank_train, rank_train_label,
                    batch_size=BATCH_SIZE*8, 
                    epochs=NEPOCH,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stop,checkpoint])

Epoch 1/50
269/269 [==============================] - 48s 166ms/step - loss: 0.4634 - val_loss: 0.1879
Epoch 2/50
269/269 [==============================] - 43s 161ms/step - loss: 0.4130 - val_loss: 0.1607
Epoch 3/50
269/269 [==============================] - 43s 161ms/step - loss: 0.3968 - val_loss: 0.1471
Epoch 4/50
269/269 [==============================] - 43s 161ms/step - loss: 0.3891 - val_loss: 0.1425
Epoch 5/50
269/269 [==============================] - 43s 162ms/step - loss: 0.3850 - val_loss: 0.1379
Epoch 6/50
269/269 [==============================] - 43s 161ms/step - loss: 0.3830 - val_loss: 0.1387
Epoch 7/50
269/269 [==============================] - 43s 161ms/step - loss: 0.3789 - val_loss: 0.1370
Epoch 8/50
269/269 [==============================] - 43s 161ms/step - loss: 0.3744 - val_loss: 0.1357
Epoch 9/50
269/269 [==============================] - 43s 161ms/step - loss: 0.3717 - val_loss: 0.1335
Epoch 10/50
269/269 [==============================] - 43s 162ms/step - l

In [ ]:
rank_model.load_weights('./rank_checkpoint/')

In [34]:
del rank_train
gc.collect()

3380

In [35]:
pred = rank_model.predict(rank_test)

In [36]:
pred_df = pd.DataFrame(columns=['user_id','movie_id','pred'])
pred_df['user_id'] = rank_test['user_id']
pred_df['movie_id'] = rank_test['movie_id']
pred_df['pred'] = pred

pred_df = pred_df.sort_values(by=['user_id','pred'], ascending=False).reset_index(drop=True)
pred_df = pred_df.groupby('user_id')['movie_id'].apply(list).reset_index()

In [37]:
test_label_df = pd.DataFrame(columns=['user_id','label'])
test_label_df['user_id'] = match_test['user_id']
test_label_df['label'] = match_test_label.tolist()

In [38]:
test_label_df = pd.merge(test_label_df, pred_df, on=['user_id'], how='left')

In [ ]:
map_at_k(test_label_df['label'], test_label_df['movie_id'], k=10)

In [ ]:
recall_at_k(test_label_df['label'], test_label_df['movie_id'], k=10)

In [ ]:
recall_at_k(test_label_df['label'], test_label_df['movie_id'], k=100)